In [1]:
import sys
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession.builder \
    .getOrCreate()



In [18]:
# #### IGNORE THIS CELL -- SCRATCH


# # 4 -> Issuer Date
# # 21 -> Borough
# # 23 -> House Number
# # 24 -> Street Label


# def initFilter(items):
# #     row = items.split(',')
#     row = items.strip('][').split(',')
#     # If len is right, and not a header, and not empty, and valid year/borough
#     return len(row) >= 25 and not row[0] == "Summons Number" and not (
#                 (row[4] == "" or row[4] == None or row[4] == " ") or
#                 (row[21] == "" or row[21] == None or row[21] == " ") or
#                 (row[23] == "" or row[23] == None or row[23] == " ") or
#                 (row[24] == "" or row[24] == None or row[24] == " ")
#             ) and (
#                 row[4][-4:] in ['2015', '2016', '2017', '2018', '2019']
#             ) and (
#                 row[21] in ['MAN','MH','MN','NEWY','NEW Y','NY','BRONX','BX','BK','K','KING','KINGS','Q','QN','QNS','QU','QUEEN','R','RICHMOND']
#             )

# def simplifyData(item):
#     row = item.strip('][').split(',')
#     year = row[4][-4:]
#     borough = row[21]
#     lowerStreet = row[24].lower()

#     if borough in ['MAN','MH','MN','NEWY','NEW Y','NY']:
#         return (year, 'M', row[23], lowerStreet)
#     elif borough in ['BRONX','BX']:
#         return (year, 'BX', row[23], lowerStreet)
#     elif borough in ['BK','K','KING','KINGS']:
#         return (year, 'K', row[23], lowerStreet)
#     elif borough in ['Q','QN','QNS','QU','QUEEN']:
#         return (year, 'Q', row[23], lowerStreet)
#     elif borough in ['R','RICHMOND']:
#         return (year, 'R', row[23], lowerStreet)


# rdd = sc.textFile('Parking_Violations_Issued*.csv')

# rdd \
# .filter(initFilter) \
# .map(simplifyData) \
# .take(30)



def checkInRange(a, b, c):
    la = len(a)
    lb = len(b)
    lc = len(c)
    # If unequal len not even
    if la != lb or la != lc or la is None or lb is None or lc is None:
        return False

    for i in range(la):
        # If a part doesn't match, not even
        # If we can't handle, skip

        if not ( int(b[i]) <= int(a[i]) and int(a[i]) <= int(c[i]) ):
            return False
    # Otherwise the houses matched
    return True

checkInRange(['82', '46'],['82','027'],['82','051'])

True

In [2]:
##### THIS CURRECT BEST VERSION

def filterInitData(row):
    # If len is right, and not a header, and not empty, and valid year/borough
    return len(row) >= 25 and not row[0] == "Summons Number" and not (
                (row[4] == "" or row[4] == None or row[4] == " ") or
                (row[21] == "" or row[21] == None or row[21] == " ") or
                (row[23] == "" or row[23] == None or row[23] == " ") or
                (row[24] == "" or row[24] == None or row[24] == " ")
            ) and (
                row[4][-4:] in ['2015', '2016', '2017', '2018', '2019']
            ) and (
                row[21] in ['MAN','MH','MN','NEWY','NEW Y','NY','BRONX','BX','BK','K','KING','KINGS','Q','QN','QNS','QU','QUEEN','R','RICHMOND']
            )

def simplifyData(row):
#     return row
    year = int(row[4][-4:])
    borough = row[21]
    houseNum = row[23]
    uStreet = row[24].upper()
    
    # If any of the houseNum is no a digit, remove it, of it no house Number
    if houseNum == '' or houseNum == ' ' or houseNum == None:
        return None # Can't stack this into one OR, because OR checks both and errors
    # Verify that each bit is value. If number for example:
    # 256A is valid. But an address like A is not valid.
    # <number><non-numer> valid given non-number is - or otherwise removed
#     if not all([char.isdigit() or char == '-' for char in houseNum]):
#         return None
    
    ##### NOTE IN THE DATASET FOR MATCHING, TH KILLS THE MATCH. EX: W 8TH ST, TH CAUSES NO MATCHES
    ##### ALSO, OUR CENTERLINE DATASET NEEDS TO BE STRIPPED OF SPACES BECAUSE OF EXTRA SPACES. SAME HERE.
        ### EX: E  8 ST: IS CENTERLINE. BUT HERE, IT'S: E 8TH ST. WON'T MATCH
    newStr = ""
    for part in uStreet.split():
        if part[0].isdigit() and (part[-2:] == 'TH' or part[-2:] == 'ST' or part[-2:] == 'ND'):
            newStr += part[:-2]
        else:
            newStr += part
    uStreet = newStr
    
    ### House number can also be in the form xxxx-xxxxx
    ### So we need to handle that and return as a list
    houseNum = houseNum.split('-')
    
    # Convert each houseNum to int
    for i, num in enumerate(houseNum): # For each split bit
        # If empty or only 1 digit
        if len(num) == 0 or (len(num) == 1 and not houseNum[0].isdigit()):
            return None
        # Make each num in houseNum their new version
        elif len(num) >= 2 and not num[-1].isdigit():
            houseNum[i] = num[:-1]
        else:
            houseNum[i] = num
        
#     houseNum = [int(num) if num[-1].isdigit() else int(num[:-1]) for num in houseNum]
        
    # BOROUGH RETURNED IN FORM SEEN ON BOROCODE IN CENTERLINE
    if borough in ['MAN','MH','MN','NEWY','NEW Y','NY']:
        return (year, 1, houseNum, uStreet)
    elif borough in ['BRONX','BX']:
        return (year, 2, houseNum, uStreet)
    elif borough in ['BK','K','KING','KINGS']:
        return (year, 3, houseNum, uStreet)
    elif borough in ['Q','QN','QNS','QU','QUEEN']:
        return (year, 4, houseNum, uStreet)
    elif borough in ['R','RICHMOND']:
        return (year, 5, houseNum, uStreet)

spark = SparkSession.builder \
    .getOrCreate()

df = spark.read.format('csv') \
                .option('header',True) \
                .option('sep', ',') \
                .option('multiLine', True) \
                .load('Parking_Violations_Issued_-_Fiscal_Year_2015.csv')

rdd = df.rdd.map(tuple)

rdd = rdd.filter(filterInitData).map(simplifyData).filter(lambda item: item is not None and item[0] is not None and item[1] is not None and item[3] is not None).cache()


##### NOTE IN THE DATASET FOR MATCHING, TH KILLS THE MATCH. EX: W 8TH ST, TH CAUSES NO MATCHES
##### ALSO, OUR CENTERLINE DATASET NEEDS TO BE STRIPPED OF SPACES BECAUSE OF EXTRA SPACES. SAME HERE.


#### DONT FORGET TO STRIP STREET LABEL AND FULL STREE TOO !!!!!!!

# Total count: 5077625
# Total Count: 4332278

# rdd = sc.textFile("Parking_Violations_Issued*.csv")
# rdd = rdd.map(initData)
rdd.take(50)

[(2015, 1, ['158'], '8AVE'),
 (2015, 1, ['10'], 'E29ST'),
 (2015, 2, ['115'], 'W172ST'),
 (2015, 1, ['350'], 'W58ST'),
 (2015, 4, ['8027'], 'JAMAICAAVE'),
 (2015, 4, ['82', '46'], '135ST'),
 (2015, 4, ['82', '46'], '135ST'),
 (2015, 1, ['68'], 'KINGST'),
 (2015, 2, ['1836'], 'BRONXDALEAVE'),
 (2015, 1, ['64'], 'W70ST'),
 (2015, 4, ['214', '47'], 'JAMAICAAVE'),
 (2015, 1, ['1534'], '2AVE'),
 (2015, 2, ['2311'], 'ARTHURAVE'),
 (2015, 2, ['1669'], 'GARFIELDST'),
 (2015, 4, ['22', '60'], '26ST'),
 (2015, 3, ['12'], 'GRAHAMAVE'),
 (2015, 1, ['264'], 'BROOMEST'),
 (2015, 1, ['264'], 'BROOMEST'),
 (2015, 1, ['44'], 'BEAVERST'),
 (2015, 2, ['250'], 'BEDFORDPARKBLVD'),
 (2015, 1, ['101'], 'DELANCEYST'),
 (2015, 4, ['40', '18'], '29ST'),
 (2015, 4, ['89', '09'], '162ST'),
 (2015, 4, ['42', '02'], 'COLLEGEPOINTBLVD'),
 (2015, 3, ['860'], 'PARKAVE'),
 (2015, 1, ['130'], 'E75ST'),
 (2015, 4, ['156', '08'], 'CROSSBAYBLVD'),
 (2015, 2, ['137'], 'W179ST'),
 (2015, 2, ['137'], 'W179ST'),
 (2015, 1, ['1

In [10]:
###### CLEAN THE DATA. Maybe save this & send to our Database.

import json

data = ""
with open('NYC Street Centerline (CSCL).geojson', 'r') as f:
    data = json.loads(f.read())

# Strip data
for i in range(len(data['features'])):
    try:
        if data['features'][i]['properties']['full_stree'] is None or data['features'][i]['properties']['st_label'] is None or data['features'][i]['properties']['borocode'] is None:
            data['features'][i] = None
    except:
        data['features'] = None
    data['features'][i]['properties']['full_stree'] = ''.join(data['features'][i]['properties']['full_stree'].split())
    data['features'][i]['properties']['st_label'] = ''.join(data['features'][i]['properties']['st_label'].split())
    del data['features'][i]['geometry']
    del data['features'][i]['type']
    
    # Split any '-' for low, high, r and l
    try:
        data['features'][i]['properties']['l_low_hn'] = [int(i) for i in data['features'][i]['properties']['l_low_hn'].split('-')]
        data['features'][i]['properties']['r_low_hn'] = [int(i) for i in data['features'][i]['properties']['r_low_hn'].split('-')]
        data['features'][i]['properties']['r_high_hn'] = [int(i) for i in data['features'][i]['properties']['r_high_hn'].split('-')]
        data['features'][i]['properties']['l_high_hn'] = [int(i) for i in data['features'][i]['properties']['l_high_hn'].split('-')]
    except: # For any other case just continue
        data['features'][i] = None

data['features'] = [point for point in data['features'] if point is not None]

with open('modified.json', 'w') as f:
    json.dump(data, f)


In [8]:
#### COUPLE ASSUMPTIONS ABOOUT DAA
# L HIGH AND LOW ARE IN LIST FORM, SEPERATED BY -
# THE FULL STREET ADDR AND STREET LABEL ARE CAPS AND CONCATTED

# Now let's actually do physical ID matching

# First broadcast
centerlineB = sc.broadcast(data)

# Here is our function for matching phsyical ID else return None. Then we'll filter Nones
def matchPhysID(item):
    year = item[0]
    boro  = item[1]
    
    houseNum = item[2]
    # One known case that fails, human data entry error. Ex: "99-15"
    try:
        houseOdd = int(houseNum[-1]) & 1
    except:
        return None
    
    st = item[3]
    physID = None
    
    # Check A is between B and C; B < A < C
    # This checks each part. Because 123-456 is valid address
    def checkInRange(a, b, c):
        la = len(a)
        lb = len(b)
        lc = len(c)
        # If unequal len not even
        if la != lb or la != lc or la is None or lb is None or lc is None:
            return False
        ai, bi, ci = 0, 0, 0

        for i in range(la):
            # If a part doesn't match, not even
            # If we can't handle, skip
            try:
                ai = int(a[i])
                bi = int(b[i])
                ci = int(c[i])
            except:
                return False
        
            if not ( bi <= ai and ai <= ci ):
                return False
        # Otherwise the houses matched
        return True

    c_data = centerlineB.value
    
    for c_item in c_data['features']:
        # Street in full_stree
        # Street in street_label
        # Borough code matches
        if (st == c_item['properties']['full_stree'] or
        st == c_item['properties']['st_label']) and boro == int(c_item['properties']['borocode']): 
            # If we are here, confirms that street exists in the right boro
            # Now check whether house is right
            if houseOdd: # If odd house check L
                if checkInRange(houseNum, c_item['properties']['l_low_hn'], c_item['properties']['l_high_hn']): # If num is in the range, GOOD
                    physID = c_item['properties']['physicalid']
                    break
            else: # If even check R
                if checkInRange(houseNum, c_item['properties']['r_low_hn'], c_item['properties']['r_high_hn']): # If num is in range, GOOD
                    physID = c_item['properties']['physicalid']
                    break
    return (physID, year, boro)

# Get our Physical IDs and kill if None
rdd.map(matchPhysID).filter(lambda x: x is not None and x[0] is not None).take(20)


[('1109', 2015, 1),
 ('95952', 2015, 1),
 ('95237', 2015, 2),
 ('16760', 2015, 1),
 ('81255', 2015, 4),
 ('81255', 2015, 4),
 ('79680', 2015, 1),
 ('42113', 2015, 2),
 ('181502', 2015, 1),
 ('6372', 2015, 4),
 ('2158', 2015, 1),
 ('54349', 2015, 2),
 ('50536', 2015, 2),
 ('22810', 2015, 4),
 ('171247', 2015, 3),
 ('4035', 2015, 1),
 ('4035', 2015, 1),
 ('139', 2015, 1),
 ('38071', 2015, 2),
 ('4181', 2015, 1)]